RAG - REACT - TOOL

In [7]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True) 

True

In [58]:
from langchain.prompts import PromptTemplate
from langchain import hub
from langchain.agents import Tool, AgentExecutor, initialize_agent, create_react_agent
#from langchain.tools import DuckDuckGoSearchRun, WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain_openai import ChatOpenAI
#from langchain.chains import LLMMathChain, LLMChain
import spacy

# Create tools for the agent
################### first custom tool #############

#create the WMI serach tool
#Load the documents
from langchain_community.document_loaders import DirectoryLoader
DATA_PATH = "mydata"
loader = DirectoryLoader(DATA_PATH, glob="*.pdf")
docs = loader.load()

#Split the documents
from langchain.text_splitter import RecursiveCharacterTextSplitter
#from langchain.text_splitter import SentenceTransformersTokenTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
#text_splitter = SentenceTransformersTokenTextSplitter(
    chunk_size=1000,
    chunk_overlap=50,
)
documents = text_splitter.split_documents(docs)


## This does not work yet ####
"""
import spacy
nlp = spacy.load("en_core_web_sm")
#import en_core_web_sm
#nlp = en_core_web_sm.load()
doc = nlp(docs)
"""


##Get teh chunks
#chunks = text_splitter.create_documents([docs])
#print(f'Now you have {len(chunks)}')
## End of getting chinks

'''
print("####### FIRST 10 docs ############")
for i in range(10):
    print("\n")
    print(documents[i])

print("####### LAST 10 docs ############")
x = documents[::-1]
i = 0
while (i < 10):
    print("\n")
    print(x[i])
    i+= 1

'''

print("############## # of Documents ##################")
print(len(documents))


############## # of Documents ##################
320


In [59]:
#Initiatlise the Embeddings
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()
#embeddings = OpenAIEmbeddings(model='text-embedding-3-small', dimensions=1536)

In [60]:
# I'm importing the necessary libraries and initializing the Pinecone client
import os
import pinecone

from langchain_community.vectorstores import Pinecone

pc = pinecone.Pinecone()

In [61]:
# deleting all indexes
indexes = pc.list_indexes().names()
for i in indexes:
    print('Deleting all indexes ... ', end='')
    pc.delete_index(i)
    print('Done')

Deleting all indexes ... Done


In [62]:
# creating an index
from pinecone import ServerlessSpec
#from pinecone import PodSpec
index_name = 'wmsearch-openai'
if index_name not in pc.list_indexes().names():
    print(f'Creating index {index_name}')
    
    pc.create_index(
        name=index_name,
        dimension=1536,
        metric='cosine',
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        ) 
    )

    ### Code for GCP starter ####
    """
    # creating a new index
    pc.create_index(
        name=index_name,
        dimension=1536,
        metric='cosine',
        spec=PodSpec(
            environment='gcp-starter'
        )
    )
    """


    print('Index created! 😊')
else:
    print(f'Index {index_name} already exists!')

Creating index wmsearch-openai
Index created! 😊


####  Calculate Embedding cost  #####

In [64]:
# calculate embedding cost using tiktoken
def calculate_embedding_cost(texts):
    import tiktoken
    enc = tiktoken.encoding_for_model('text-embedding-3-small')
    total_tokens = sum([len(enc.encode(page.page_content)) for page in texts])
    # check prices here: https://openai.com/pricing
    # print(f'Total Tokens: {total_tokens}')
    # print(f'Embedding Cost in USD: {total_tokens / 1000 * 0.00002:.6f}')
    return total_tokens, total_tokens / 1000 * 0.00002

tokens, embedding_cost = calculate_embedding_cost(documents) ##documents == chunks

print(f'Total Tokens: {tokens}')
print(f'Embedding Cost in USD: {tokens / 1000 * 0.00002:.6f}')

Total Tokens: 60972
Embedding Cost in USD: 0.001219


In [66]:
# processing the input documents, generating embeddings using the provided `OpenAIEmbeddings` instance,
# inserting the embeddings into the index and returning a new Pinecone vector store object. 
vectorstore = Pinecone.from_documents(documents, embeddings, index_name=index_name)

index = pc.Index(index_name)
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 640}},
 'total_vector_count': 640}

In [67]:
from typing import Optional
from langchain.tools import BaseTool
from langchain.callbacks.manager import AsyncCallbackManagerForToolRun, CallbackManagerForToolRun

class WMSearchTool(BaseTool):
    name = "WM search"
    description = "useful when you need to answer questions about WM product"

    def _run(self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None) -> str:
        #store = vectorstore.as_retriever()
        store = vectorstore.as_retriever(search_type='similarity', search_kwargs={'k': 5})  
        #docs = store.get_relevant_documents(query)
        docs = store.invoke(query)
        text_list = [doc.page_content for doc in docs]
        return "\n".join(text_list)

    '''
    async def _arun(self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("custom_search does not support async")
    '''

################## custom tool end ####################



In [68]:
# 1. Python REPL Tool (for executing Python code)
from langchain.prompts import PromptTemplate
from langchain import hub
from langchain.agents import Tool, AgentExecutor, initialize_agent, create_react_agent
from langchain.tools import DuckDuckGoSearchRun, WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain_openai import ChatOpenAI
from langchain.chains import LLMMathChain, LLMChain

from langchain_openai import OpenAI

llm=OpenAI()

python_repl = PythonREPLTool()
python_repl_tool = Tool(
    name='Python REPL',
    func=python_repl.run,
    description='Useful when you need to use Python to answer a question. You should input Python code.'
)

In [69]:
from langchain.prompts import PromptTemplate
from langchain import hub
from langchain.agents import Tool, AgentExecutor, initialize_agent, create_react_agent
from langchain.tools import DuckDuckGoSearchRun, WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain_openai import ChatOpenAI
from langchain.chains import LLMMathChain, LLMChain

from langchain_openai import OpenAI

llm=OpenAI()

# 2. Wikipedia Tool (for searching Wikipedia)
api_wrapper = WikipediaAPIWrapper()
wikipedia = WikipediaQueryRun(api_wrapper=api_wrapper)
wikipedia_tool = Tool(
    name='Wikipedia',
    func=wikipedia.run,
    description='Useful for when you need to look up a topic, country, or person on Wikipedia.'
)

# 3. DuckDuckGo Search Tool (for general web searches)
search = DuckDuckGoSearchRun()
duckduckgo_tool = Tool(
    name='DuckDuckGo Search',
    func=search.run,
    description='Useful for when you need to perform an internet search to find information that another tool can\'t provide.'
)

#Another math tool as REPL not working as expected
from langchain.chains import LLMMathChain, LLMChain
problem_chain = LLMMathChain.from_llm(llm=llm)
math_tool = Tool.from_function(name="Calculator",
                func=problem_chain.run,
                description="Useful for when you need to answer questions about math. This tool is only for math questions and nothing else. Only input math expressions."
)



In [70]:
from langchain.prompts import PromptTemplate
from langchain import hub
from langchain.agents import Tool, AgentExecutor, initialize_agent, create_react_agent
from langchain.tools import DuckDuckGoSearchRun, WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain_openai import ChatOpenAI
from langchain.chains import LLMMathChain, LLMChain
from langchain.memory import ConversationBufferMemory

from langchain_openai import OpenAI

#llm=OpenAI()

# Initialize the ChatOpenAI model (gpt-4-turbo-preview) with a temperature of 0. Utilize gpt-3.5-turbo if you use the free plan
llm = ChatOpenAI(model_name='gpt-4-turbo', temperature=0)

#Answer the following questions in GERMAN as best you can.
# Define a template for answering questions
template = '''
Answer the following questions as best you can.
if you don't know the answer, say "I don't know".
Answer in Bengali
Questions: {q}
'''

#Answer in Kannada Language. # just for fun

# Create a PromptTemplate object from the template
prompt_template = PromptTemplate.from_template(template)

# Pull the react prompt from the hub
prompt = hub.pull('hwchase17/react')


# displaying information about the react prompt
# print(type(prompt))
# print(prompt.input_variables)
# print(prompt.template)


# Combine the tools into a list
tools = [WMSearchTool(), math_tool, python_repl_tool, wikipedia_tool, duckduckgo_tool]

# Create a react agent with the ChatOpenAI model, tools, and prompt
agent = create_react_agent(llm, tools, prompt)

memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

# Initialize the AgentExecutor
agent_executor = AgentExecutor(
    agent=agent,
    memory=memory,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=5
)


In [71]:
#question = 'Does WMi support json interfaces?'
#question = 'Who is the current prime minister of the UK?'
#question = 'what is the answer to 3 ** 3.9'
#question = 'Tell me about Napoleon Bonaparte early life'
#question = 'how to enable "Putaway To Empty Non-Replenishable Dynamic Active Locations" in Manhattan Active Supply Chain'
#question = 'how to enable "Putaway To Empty Non-Replenishable Dynamic Active Locations" in Manhattan Active® Warehouse Management?'
#question = 'Is the feature "Creation of Multiple Process Needs for an Item" in Manhattan Active Supply Chain, enabled by default?'
#question = 'Is the feature "Creation of Multiple Process Needs for an Item" in Manhattan Active® Warehouse Management, enabled by default?'
question = 'What is the maximum weight and volume supported for transportaion order size in Manhattan Active TM?'
#question = 'what is the default maximum weight and volume supported?'
#question = 'what is the default maximum weight supported in lbs?'
#question = 'Explain transportation order alerts and missing fields'

output = agent_executor.invoke({
    'input': prompt_template.format(q=question)
})

print(output['output'])



> Entering new AgentExecutor chain...
I need to find specific information about the maximum weight and volume supported for transportation order size in Manhattan Active TM.

Action: WM search

Action Input: "maximum weight and volume supported for transportation order size in Manhattan Active TM"


Error in LangChainTracer.on_tool_end callback: TracerException("Found chain run at ID ee228d16-dc90-456b-ba7c-60366202b491, but expected {'tool'} run.")


Manhattan Active TM allows users to subscribe to and receive notifications when the planned sizes of transportation orders exceed

the maximum capacity allowed within the solution. Currently, the default maximum weight of 40000 lbs. (standard weight UOM) and

maximum volume of 40000 cuft (standard volume UOM) are set. If the user needs to reset, then modify the rule using the sample

rule “(transportationOrder.totalWeight > 40000) or (transportationOrder.totalVolume > 40000)”, which refers to the Alert Type

“TOSizeMaximumCapacityViolation.”

Enablement: This feature is controlled by a configuration that is disabled by default.

Related:

Transportation Management Alerts

Non-Event-Based Notification

Manhattan Active TM supports the following shipment non-event-based notifications:

Shipment in tendered status but not accepted by the carrier.

Shipment in planned status but not tendered.

Shipment missed the scheduled appointment.
Manhattan Active TM allows users to subscribe to and r

In [72]:
for item in output['chat_history']:
    print(item)

content='\nAnswer the following questions as best you can.\nif you don\'t know the answer, say "I don\'t know".\nAnswer in Bengali\nQuestions: What is the maximum weight and volume supported for transportaion order size in Manhattan Active TM?\n'
content='ম্যানহাটন অ্যাক্টিভ টিএম-এ পরিবহনের অর্ডারের জন্য সর্বাধিক ওজন ৪০,০০০ পাউন্ড এবং সর্বাধিক আয়তন ৪০,০০০ ঘনফুট সমর্থন করে।'
